In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State

# Plotting & Data
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import base64
import os

# My custom CRUD module
from crud_class import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#My unique info to access database
USER = 'aacuser'
PASS = 'jacquelinkitcher'
HOST = 'nv-desktop-services.apporto.com'
PORT = 33975
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD module
shelter = AnimalShelter(USER, PASS, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read_in({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

# app instance
app = JupyterDash(__name__)

# To display logo in database
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    #This will include the link to the SNHU website when the logo is clicked.
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
            href="https://www.snhu.edu/"
        ),
        #This is my personalization
        html.P("Jacquelin Kitcher"),
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.Hr(),

        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            editable=False,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable=False,
            row_selectable=False,
            row_deletable=True,
            style_header={
                "backgroundColor": "#4CAF50",
                "color": "white",
                "fontWeight": "bold"
            },
            selected_columns=[],
            selected_rows=[0],
            page_action="native",
            page_current=0,
            page_size=10
        ),

        html.Br(),
        html.Hr(),
        #Dropdown menu for animal type
        dcc.Dropdown(
            id='filter-type',
            options=[{'label': i, 'value': i} for i in df['animal_type'].unique()],
            placeholder="Select an Animal Type",
            style={'width': '50%', 'margin': '10px'}
        ),

        html.Div(
            className='row',
            style={'display': 'flex'},
            children=[
                html.Div(id='graph-id', className='col s12 m6'),
                html.Div(id='map-id', className='col s12 m6')
            ]
        )
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################   
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type:
        filtered_df = df[df['animal_type'] == filter_type]
    else:
        filtered_df = df
    return filtered_df.to_dict('records')

#Callback for breed pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available to display chart.")]
    
    dff = pd.DataFrame(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

#Callback to highlight cell in database when clicked.
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#Callback for geolocation feature
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData or not index:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

#Run code to display database
app.run_server(mode='inline', debug=True)